In [1]:
import sys
sys.path.append('../..')
from tools.GA.NumericalGeneticAlgorithm import NumericalGeneticAlgorithm
from sklearn.model_selection import train_test_split
from weka.classifiers import Classifier, Evaluation
import weka.core.converters as converters
from weka.core.classes import Random
from weka.filters import Filter
import pandas as pd
import numpy as np
import csv
import weka.core.jvm as jvm

In [2]:
jvm.start(system_cp=True, packages=True, max_heap_size="512m")

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Adding system classpath
DEBUG:weka.core.jvm:Classpath=['C:\\Users\\ch\\miniconda3\\envs\\tf\\Lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'C:\\Users\\ch\\miniconda3\\envs\\tf\\Lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'C:\\Users\\ch\\miniconda3\\envs\\tf\\Lib\\site-packages\\javabridge\\jars\\cpython.jar', 'c:\\Users\\ch\\miniconda3\\envs\\tf\\lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'c:\\Users\\ch\\miniconda3\\envs\\tf\\lib\\site-packages\\weka\\lib\\core.jar', 'c:\\Users\\ch\\miniconda3\\envs\\tf\\lib\\site-packages\\weka\\lib\\mtj.jar', 'c:\\Users\\ch\\miniconda3\\envs\\tf\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'c:\\Users\\ch\\miniconda3\\envs\\tf\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=512m
DEBUG:weka.core.jvm:Package support enabled


In [3]:
rnd_seed = 42
runs=10
nominal_filter = Filter(classname="weka.filters.unsupervised.attribute.NumericToNominal", options=["-R", "last"])

In [4]:
data_set="..\\datasets\\Numerical\\sonar.csv"

In [ ]:
data = converters.load_any_file(data_set)

data.class_is_last()

nominal_filter.inputformat(data)
data = nominal_filter.filter(data)


cls = Classifier(classname="weka.classifiers.trees.J48")

evaluation = Evaluation(data)
evaluation.crossvalidate_model(cls, data, 10, Random(42))
print(evaluation.summary())


Correctly Classified Instances         151               72.9469 %
Incorrectly Classified Instances        56               27.0531 %
Kappa statistic                          0.4553
Mean absolute error                      0.2703
Root mean squared error                  0.507 
Relative absolute error                 54.3269 %
Root relative squared error            101.6491 %
Total Number of Instances              207     



In [ ]:
splits=[]
for s in range(10,100,10):
    train, test = train_test_split(data, random_state=42, test_size=s/100)

    geneticAlgorithm = NumericalGeneticAlgorithm(train=pd.DataFrame(train), test= pd.DataFrame(test))

    print("################################ test size= "+str(s)+"% ################################")
    results=[]
    
    for i in range(runs):
        results.append(geneticAlgorithm.run(
            generation_limit=50,
            population_size=10,
            rnd_seed=42
        ))
    
    splits.append(results)
    np.save('instance.npy', np.array(splits, dtype=object))

In [ ]:
np.save('Results\\sonar.npy', np.array(splits, dtype=object))
splits=np.load('Results\\sonar.npy',allow_pickle=True)

In [ ]:
best_split=0
best_data=None
best_solution=0
test_size=0
for result in splits:   
    test_size=test_size+10
    print("################################ test size= "+str(test_size)+"% ################################") 
    for i in range(runs):
        data=result[i]
        train=data[0]
        test=data[1]
        with open('train.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            for row in train:
                writer.writerow(row)

        with open('test.csv', 'w', newline='') as file:
            writer = csv.writer(file)
            for row in test:
                writer.writerow(row)

        test = converters.load_any_file("test.csv")
        test.class_is_last()
        
        nominal_filter.inputformat(test)
        test = nominal_filter.filter(test)
        
        cls = Classifier(classname="weka.classifiers.trees.J48")
        evaluation=Evaluation(test)
        evaluation.crossvalidate_model(cls, test, 10, Random(42))
        print("run number "+str(i)+" accuarcy: "+str(evaluation.percent_correct))
        if evaluation.percent_correct>best_solution:
            best_split=test_size/100
            best_data=result[i]
            best_solution=evaluation.percent_correct

################################ test size= 10% ################################
run number 0 accuarcy: 71.42857142857143
run number 1 accuarcy: 66.66666666666667
run number 2 accuarcy: 66.66666666666667
run number 3 accuarcy: 61.904761904761905
run number 4 accuarcy: 47.61904761904762
run number 5 accuarcy: 57.142857142857146
run number 6 accuarcy: 57.142857142857146
run number 7 accuarcy: 66.66666666666667
run number 8 accuarcy: 61.904761904761905
run number 9 accuarcy: 66.66666666666667
################################ test size= 20% ################################
run number 0 accuarcy: 47.61904761904762
run number 1 accuarcy: 76.19047619047619
run number 2 accuarcy: 73.80952380952381
run number 3 accuarcy: 66.66666666666667
run number 4 accuarcy: 59.523809523809526
run number 5 accuarcy: 76.19047619047619
run number 6 accuarcy: 59.523809523809526
run number 7 accuarcy: 59.523809523809526
run number 8 accuarcy: 69.04761904761905
run number 9 accuarcy: 61.904761904761905
##########

In [ ]:
data=best_data
train=data[0]
test=data[1]

In [ ]:
with open('train.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for row in train:
        writer.writerow(row)

with open('test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    for row in test:
        writer.writerow(row)

In [ ]:
test = converters.load_any_file("test.csv")
test.class_is_last()

train = converters.load_any_file("train.csv")
train.class_is_last()


nominal_filter.inputformat(train)
train = nominal_filter.filter(train)

nominal_filter.inputformat(test)
test = nominal_filter.filter(test)


In [ ]:
cls = Classifier(classname="weka.classifiers.trees.J48")
evaluation=Evaluation(test)
evaluation.crossvalidate_model(cls, test, 10, Random(42))
print("train size: "+str(1-best_split))
print("test size: "+str(best_split))
print(evaluation.summary())
print(evaluation.confusion_matrix)

train size: 0.19999999999999996
test size: 0.8

Correctly Classified Instances         135               81.3253 %
Incorrectly Classified Instances        31               18.6747 %
Kappa statistic                          0.6245
Mean absolute error                      0.2034
Root mean squared error                  0.4193
Relative absolute error                 40.7747 %
Root relative squared error             83.9486 %
Total Number of Instances              166     

[[74. 13.]
 [18. 61.]]
